In [9]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq

In [10]:
api_key_groq=os.getenv("GROQ_API_KEY")
api_nvidia_nim=os.getenv("NVIDIA_NIM_KEY")

#### Defining LLM Model (Llama3–70B)


##### 1. Using LLM through Groq

In [11]:

llm_groq=ChatGroq(temperature=0,
             model_name="llama3-70b-8192",
             api_key=api_key_groq)

#### 2. Using llm through Nvidia nim api
https://build.nvidia.com/meta/llama-3_1-8b-instruct

In [12]:
## Core LC Chat Interface
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

c:\Users\s21\Desktop\SRS-Agents\.venv\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:161: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(


[Model(id='adept/fuyu-8b', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1/vlm/adept/fuyu-8b', aliases=['ai-fuyu-8b', 'playground_fuyu_8b', 'fuyu_8b'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='microsoft/phi-3-small-8k-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-phi-3-small-8k-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='mistralai/mixtral-8x7b-instruct-v0.1', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-mixtral-8x7b-instruct', 'playground_mixtral_8x7b', 'mixtral_8x7b'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='01-ai/yi-large', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-yi-large'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='deepseek-ai/deepseek-coder-6.7b-instruct', model_type='chat', client='ChatN

In [14]:
## Core LC Chat Interface
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm_nv = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", api_key=api_nvidia_nim, temperature=0, max_tokens=12000)
llm_nv

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='meta/llama-3.1-8b-instruct', temperature=0.0, max_tokens=12000)

##### Use any llm 

In [47]:
llm=llm_groq

In [48]:
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
)

writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

editor = Agent(
    llm=llm,    
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [49]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [50]:
crew= Crew(
   agents=[planner, writer, editor],
   tasks=[plan, write, edit],
   manager_llm=llm,
   process=Process.sequential,
   verbose=True
)

2024-09-05 14:38:34,587 - 3356 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
result = crew.kickoff(inputs={"topic": "Gen AI "})

In [54]:
from IPython.display import Markdown
Markdown(result)

**Unlocking the Power of Gen AI: Trends, Players, and Insights**
============================================================

**Introduction**
---------------

Artificial intelligence (AI) has come a long way since its inception, transforming from a futuristic concept to a ubiquitous technology that permeates every aspect of our lives. As AI continues to evolve, a new generation of intelligent systems is emerging – Gen AI, or Artificial General Intelligence. In this article, we'll delve into the world of Gen AI, exploring the latest trends, key players, and noteworthy developments that are shaping the future of AI.

Gen AI refers to AI systems that possess the ability to understand, learn, and apply knowledge across a wide range of tasks, much like human intelligence. As we navigate the complexities of the digital landscape, understanding Gen AI is crucial for business leaders, IT professionals, and individuals interested in AI and technology.

**Latest Trends in Gen AI**
---------------------------

### Advancements in NLP and Computer Vision

Recent breakthroughs in natural language processing (NLP) and computer vision have enabled Gen AI systems to better understand and interact with humans. For instance, advancements in NLP have led to the development of more sophisticated chatbots and virtual assistants, while improvements in computer vision have enabled AI systems to recognize and interpret visual data with greater accuracy. These advancements have far-reaching implications for industries such as customer service, healthcare, and finance.

### Rise of Autonomous Systems and Edge AI

The proliferation of IoT devices has given rise to autonomous systems and edge AI, which enable real-time processing and decision-making at the edge of the network. This trend is revolutionizing industries such as healthcare, finance, and transportation, where rapid decision-making is critical. For example, autonomous systems are being used in healthcare to analyze medical images and diagnose diseases more accurately.

### Increased Focus on Explainability and Transparency

As Gen AI systems become more pervasive, there is a growing need for explainability and transparency in AI decision-making. This trend is driven by the need to build trust in AI systems and ensure that they are fair, unbiased, and accountable. Researchers and developers are working to create more transparent AI systems that can provide insights into their decision-making processes.

### Growing Importance of Human-AI Collaboration

The future of work will be characterized by human-AI collaboration, where humans and machines work together to achieve common goals. This trend is already evident in industries such as healthcare, where AI is being used to augment human capabilities and improve patient outcomes. Human-AI collaboration has the potential to increase productivity, reduce errors, and improve overall efficiency.

**Key Players in Gen AI**
-------------------------

### Leading Tech Companies

Tech giants such as Google, Microsoft, and Facebook are investing heavily in Gen AI research and development. These companies are driving innovation in areas such as NLP, computer vision, and autonomous systems. For example, Google's DeepMind has made significant breakthroughs in AI research, including the development of AlphaGo, a computer program that can beat human players in Go.

### Startups and Innovators

Startups such as NVIDIA, DeepMind, and Vicarious are pushing the boundaries of Gen AI, developing new applications and use cases that are transforming industries. These startups are driving innovation in areas such as edge AI, autonomous systems, and human-AI collaboration.

### Research Institutions and Organizations

Research institutions such as MIT, Stanford, and the AI Now Institute are driving Gen AI advancements, providing a platform for collaboration and innovation. These institutions are conducting cutting-edge research in areas such as AI ethics, transparency, and accountability.

**Noteworthy News and Developments in Gen AI**
---------------------------------------------

### Recent Breakthroughs in Gen AI Research

Recent breakthroughs in Gen AI research have led to the development of more sophisticated AI systems that can learn, reason, and apply knowledge across multiple domains. For example, researchers have developed AI systems that can learn from experience and adapt to new situations.

### Government Initiatives and Regulations

Governments around the world are launching initiatives and regulations to shape the Gen AI landscape, ensuring that AI is developed and deployed in a responsible and ethical manner. For example, the European Union has launched the AI Act, which aims to promote trustworthy AI development and deployment.

### Industry-Specific Use Cases and Success Stories

Gen AI is being applied across various industries, including healthcare, finance, and education, leading to improved outcomes, increased efficiency, and reduced costs. For example, AI-powered chatbots are being used in healthcare to provide personalized patient care and improve patient engagement.

**The Future of Gen AI: Opportunities and Challenges**
---------------------------------------------------

### Potential Benefits and Risks of Gen AI

Gen AI has the potential to revolutionize industries and transform the way we live and work. However, it also poses risks such as job displacement, bias, and accountability. As Gen AI becomes more pervasive, it's essential to address these risks and ensure that AI is developed and deployed in a responsible and ethical manner.

### Ethical Considerations and Responsible AI Development

As Gen AI becomes more pervasive, it's essential to develop AI systems that are fair, transparent, and accountable. This requires a focus on ethical considerations and responsible AI development. Researchers, developers, and policymakers must work together to ensure that AI is developed and deployed in a way that benefits society as a whole.

### Preparing for the Workforce and Societal Implications

The rise of Gen AI will require significant changes to the workforce and society as a whole. It's essential to prepare for these changes by investing in education and retraining programs. Governments, educational institutions, and companies must work together to ensure that workers have the skills needed to thrive in an AI-driven economy.

**Conclusion and Call to Action**
-------------------------------

In conclusion, Gen AI is a rapidly evolving field that holds immense potential for transforming industries and improving lives. As we navigate the complexities of Gen AI, it's essential to stay informed, engaged, and committed to responsible AI development.

We encourage you to stay up-to-date with the latest trends, news, and developments in Gen AI by following industry leaders, researchers, and innovators. Additionally, we recommend exploring online courses and educational resources to deepen your understanding of Gen AI and its applications.

Together, let's unlock the power of Gen AI and create a brighter future for all.

**Resources:**

* Research papers and articles from leading AI institutions and publications
* Industry reports and market analysis from reputable firms
* News articles and press releases from key players and innovators in the Gen AI space
* Online courses and educational resources for further learning